In [10]:
## Generate a graph
using StatsBase
using Combinatorics

using Pkg; Pkg.activate(".")
using HypergraphModularity

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [6]:
n = 200
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = buildΩ(ω, α0, kmax)
## Sample
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((200,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
  E: Dict{Int64,Dict}
  D: Array{Int64}((200,)) [14, 9, 9, 12, 6, 4, 4, 13, 5, 8  …  13, 5, 4, 3, 8, 12, 3, 6, 6, 6]


In [7]:
## Run Louvains, Naive, and Faster version

# @time Zlou = Naive_HyperLouvain(H,Ω;α=α0)
@time Zlou2 = HyperLouvain(H,kmax,Ω;α=α0)

# mlou = modularity(H,Zlou,Ω,α=α0)
mlou2 = modularity(H,Zlou2,Ω,α=α0)

# This may not be exactly the same, but that's okay.
# Even a difference of 1e-14 in modularity change may lead you to make a 
# different greedy decision, but it's still a valid move.
# @show mlou, mlou2
# @show abs(mlou- mlou2)

Louvain Iteration 1
Louvain Iteration 2
  0.158904 seconds (2.05 M allocations: 106.063 MiB, 12.23% gc time)


-38784.8735706929276606691836082156654708086307437042705714702606201171875

In [8]:
## Try supernode Louvain
# Sanity check: having a singletons initialization is equivalent to naive louvain
Z = collect(1:n)
@time Znew, changed = SuperNodeStep(H,Z,kmax,Ω;α=α0)
mlou3 =  modularity(H,Znew,Ω,α=α0)

toc = 0.0022649765014648438
Louvain Iteration 1
Louvain Iteration 2
t1 = 0.18848586082458496
  0.195015 seconds (2.03 M allocations: 123.083 MiB, 14.10% gc time)


-38784.37868553470197326813622730468100741063608438707888126373291015625

In [9]:
## Now repeatedly call the SuperNodeStep until no more improvement is possible
@time Z = SuperNodeLouvain(H,kmax,Ω;α=α0)
msuper = modularity(H,Z,Ω,α=α0)

Faster SuperNode Louvain: Phase 1
Louvain Iteration 1
Louvain Iteration 2
SuperNode Louvain: Phase 2
toc = 0.0018770694732666016
Louvain Iteration 1
t1 = 0.10001039505004883
  0.256537 seconds (3.07 M allocations: 169.075 MiB, 14.72% gc time)


-38784.8735706929276606691836082156654708086307437042705714702606201171875